# Synchronisation automatique des données TAROT issues d'alertes GRB

Le but de ces programmes est de synchroniser le owncloud GRANDMA avec les images TAROT alertes Fermi. 


### Prérequis


Avoir un compte owncloud GRANDMA

### Principe général

Après avoir traité une image, le programme grenouille créera dans un répertoire un fichier texte avec pour nom le nom de l'image (exemple : IM.fits.gz.txt) qui contiendra le chemin vers l'image (exemple : pif/paf/pouf/IM.fits.gz) : ce fichier texte s'appelera image fantôme. 

Un premier programme sur la machine tarot8 checkera à intervalle régulier s'il y a des nouvelles images fantômes dans le répertoire. S'il y en a une, elle est copiée dans le répertoire /tmp, puis le texte est lu pour obtenir le chemin vers la véritable image. L'image est ensuite copiée localement dans le dossier /copy sur la machine. A ce stade, il est possible d'effectuer un traitement de l'image (changement de nom, vérification de la calibration...) avant de l'envoyer en sftp dans le répertoire /tarot/.. sur la machine grandma0 à Cador. Enfin, la copie locale et l'image fantôme dans /tmp sont supprimée. 

Du côté de grandma0, le script de synchronisation de /tarot/.. à owncloud est appelé à intervalle régulier. 

### Connexion en ssh

Pour se connecter, il faut dans un premier temps se connecter en ssh à tarot.net.org.

Le mot de passe sera alors demandé.

### Sur tarot8

Pour se connecter à tarot8 en ssh, il faut lancer la commande :

Le mot de passe sera demandé : gw170817

Le script suivant a pour chemin : /home/grandma/scripts/fermi.py

/mnt/tarot4_data/srv_work_ros_data/images/ghost_images/alert_fermi est le répertoire qui contient les images fatômes.

/mnt/tarot4_data/srv_work_ros_data/images/pretraite/im/ est le répertoire qui contient les images véritable. Il faut lui ajouter le chemin relatif AAAA/MM/JJ/nom.fits.gz pour obtenir le chemin absolu vers l'image. 

In [ ]:
#!/usr/bin/env python3
# coding: utf-8

import os
import paramiko
import shutil

def connect(hostname, username, password):
        """Connects to host with username and password
        
        :hostname: str
            name of the ssh server to connect to; eg: 'ssh.tarotnet.org'
        :username: str
        :password: str
        
           """
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.load_system_host_keys()
        client.connect(hostname, username=username, password=password)
        return client

for file in os.listdir("/mnt/tarot4_data/srv_work_ros_data/images/ghost_images/alert_fermi"): 
    
    if file.startswith('.') == False:
        
        shutil.copy("/mnt/tarot4_data/srv_work_ros_data/images/ghost_images/alert_fermi/"+file,"/tmp")
        
        name = os.path.splitext(file)[0]
        print("this is the name of the file:", name)
    
        with open("/tmp/"+file, "r") as text:
            texte = text.readlines()
    
        relative_path = texte[0]
        relative_path = relative_path.strip('\n')
        
        absolute_path = "/mnt/tarot4_data/srv_work_ros_data/images/pretraite/im/" + relative_path
        print('this is the path of the file:', absolute_path)
        
        destination = "/home/grandma/copy/"
        shutil.copy(absolute_path,destination)
    
        client = connect(hostname='192.168.20.70', username='grandma',password='6lhWF"Pn')
        sftp = client.open_sftp()
        
        sftp.put("/home/grandma/copy/"+name, "/home/grandma/tarot/alert_fermi/"+name)
        sftp.close()
        
        os.remove("/home/grandma/copy/"+name)
        os.remove("/tmp/"+file)

Pour que ce code soit appelé à intervalle régulier, il faut installer un crontab sur la machine :

Puis insérer la ligne suivante. Elle permet de faire tourner le code toutes les minutes. 

Pour quitter la cellule, "ctrl + X", puis enregistrer avec Y.

### Sur grandma0

Dans un nouvel onglet du terminal, il faut dans un premier temps se reconnecter à tarotnet.org. Ensuite, pour se connecter à grandma0, saisir :

Le mot de passe sera demandé : 6lhWH"Pn

Le script suivant est de chemin : /home/grandma/scripts/script.sh

Il permet de synchroniser le owncloud grandma https://grandma-owncloud.lal.in2p3.fr avec les dossiers inclus dans /home/grandma, grâce à la fonction owncloudcmd incluse dans le client owncloud pour linux.

Le paramètre --exlude permet de définir quels dossiers de la machine grandma0 ne doivent pas être synchronisés. Il doit être placé avant le chemin vers le fichier texte exclusion_file qui contient ligne par ligne les répertoires à ne pas synchroniser. Il contient actuellement une seule ligne, "scripts", puisqu'on ne veut synchroniser que le répertoire /tarot qui contient les images qu'on a copié précédemment. 

Le paramètre --unsyncedfolders permet de définir quels dossiers sur owncloud ne doivent pas être synchronisés. Il est placé devant le fichier texte unsynced_folders qui contient ligne par ligne le nom des répertoires ou des fichiers du cloud à ne pas synchroniser. Le fichier texte contient notamment une ligne "Candidates", "Documents", ...

Sans oublier de remplacer "user" et "password" par celui d'un compte grandma.

In [ ]:
#!/bin/bash
check_PID=$(pgrep owncloudcmd)
if [ -z "$check_PID" ]
then
    owncloudcmd -s --exclude /home/grandma/scripts/exclusion_file --unsyncedfolders /home/grandma/scripts/unsynced_folders -u user -p 'password' /home/grandma https://grandma-owncloud.lal.in2p3.fr
fi

Enfin, pour que ce code soit appelé à intervalle régulier, il faut installer un crontab sur la machine grandma0:

Puis la ligne suivante : 

Puis quitter et enregistrer. 

## Voilà